In [1]:
!pip install emoji


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/608.4 kB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 10.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install -q "transformers[tf]==4.40.2" "tokenizers==0.19.1" "sentencepiece>=0.1.99" "huggingface_hub>=0.23.0" emoji



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # set sebelum import tensorflow

import tensorflow as tf
from transformers import TFAlbertModel, BertTokenizerFast

import emoji

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

2025-10-07 18:02:45.739467: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-07 18:02:45.739518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-07 18:02:45.740295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def clean_non_alphanumeric(text):
    # Define a regex pattern to match @USER format
    user_pattern = re.compile(r'@USER')
    
    # Split the text by spaces to handle each word separately
    words = text.split()
    
    # Initialize a list to hold the cleaned words
    cleaned_words = []
    
    # Loop through each word in the text
    for word in words:
        # If the word matches @USER, add it as is
        if user_pattern.fullmatch(word):
            cleaned_words.append(word)
        else:
            # Define a regex pattern to match non-alphanumeric characters
            pattern = re.compile(r'[\W_]+')  # Matches any non-alphanumeric character or underscore
            # Replace non-alphanumeric characters with a space
            cleaned_word = re.sub(pattern, ' ', word)
            cleaned_words.append(cleaned_word)
    
    # Join the cleaned words back into a single string with spaces
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text

def clean_text(text):
    # Define regex pattern to match unwanted unicode characters
    pattern = re.compile(r'[^\x00-\x7F]+')  # Matches any non-ASCII characters

    # Use sub() method to remove unwanted characters
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

def preprocess_text(text):
    # Remove "RT @username:" at the beginning of the text
    if isinstance(text, bytes):
        try:
            text = text.decode('utf-8')
        except UnicodeDecodeError:
            text = text.decode('latin-1')

    text = re.sub(r'^RT ', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    # Remove URLs
    

    # Remove emojis and other non-ASCII symbols
    text = emoji.demojize(text)

    # Convert to lowercase
    text = text.lower()
    text = re.sub(r'https?://\S+', 'HTTPURL', text)
    text = clean_text(text)
    # Remove hashtags along with their content (words starting with # followed by alphanumeric characters)
    text = re.sub(r'#\w+', '', text)

    # Remove mentions (words starting with @ followed by alphanumeric characters)
    text = re.sub(r'@\S+', '@USER', text)

    # Remove non-alphanumeric characters including remaining symbols
    text = clean_non_alphanumeric(text)

    # Remove leading and trailing spaces
    text = text.strip()

    return text


# Example usage
texts = [
    "One of my fave.. baru nyoba tapi langsung jadi favorite deh.. wajah bener2 bersih dan keset, halus dan ga kering. Kalo malem bersihin pake ini trus water mask nya, besok pagi kulit enak bangeett.. harga masih oke pula..",
    "Bau nya enak banget, aku suka.. rosey gitu tapi ga terlalu kuat.. Dia lembut, ga bikin muka kering tapi tetep besih jadi ga bikin break out.. pass aja sayangnya satu.. mahal uhuuuuy.. jadi make nya disayang sayang"
]

processed_text = [preprocess_text(text) for text in texts]
for text in processed_text:
    print(text)

one of my fave  baru nyoba tapi langsung jadi favorite deh  wajah bener2 bersih dan keset  halus dan ga kering  kalo malem bersihin pake ini trus water mask nya  besok pagi kulit enak bangeett  harga masih oke pula
bau nya enak banget  aku suka  rosey gitu tapi ga terlalu kuat  dia lembut  ga bikin muka kering tapi tetep besih jadi ga bikin break out  pass aja sayangnya satu  mahal uhuuuuy  jadi make nya disayang sayang


In [5]:
train = pd.read_csv('/kaggle/input/dataset-facialwash/dataset (3).csv', delimiter=',')

In [6]:
train

,ID Data,Nama Produk,Jenis Kulit,Review Produk,Rating,Label
0,A001,Cetaphil: Gentle Skin Cleanser,Oily,Aku suka banget sama ini. Cuma ini cleanser ya...,5,Positive
1,A002,Cetaphil: Gentle Skin Cleanser,Oily,Walaupun produknya mengklaim dapat digunakan s...,5,Positive
2,A003,Cetaphil: Gentle Skin Cleanser,Dry,Thanks God for this product. Produk ini udah p...,5,Positive
3,A004,Cetaphil: Gentle Skin Cleanser,Oily,Another favorite product from Cetaphil. Seneng...,5,Positive
4,A005,Cetaphil: Gentle Skin Cleanser,Oily,awalnya kaget karena cleanser ini enggak berbu...,5,Positive
...,...,...,...,...,...,...
495,A496,Acnes: Creamy Wash,Dry,Aku beli ini karna baca blog female daily 3 cl...,1,Negative
496,A497,Acnes: Creamy Wash,Dry,"aku pake ini udh ngabisin 1 botol, saat aku je...",1,Negative
497,A498,Acnes: Creamy Wash,Dry,Facial Wash yg aku beli pas awal2 mulai jerawa...,1,Negative
498,A499,Acnes: Creamy Wash,Dry,Aku pernah beli ini dengan harapan bisa bantu ...,1,Negative


In [7]:
print('Jumlah Data Duplikat: ', train.duplicated().sum())
train[train.duplicated() == True]

Jumlah Data Duplikat:  0


,ID Data,Nama Produk,Jenis Kulit,Review Produk,Rating,Label


In [8]:
# Informasi Missing Value
print('Jumlah Data Kosong:\n',train.isna().sum())

Jumlah Data Kosong:
 ID Data          0
Nama Produk      0
Jenis Kulit      0
Review Produk    0
Rating           0
Label            0
dtype: int64


In [9]:
print('Jumlah Awal Data: ', train.shape[0])
train.drop_duplicates(inplace=True)
print('Jumlah Setelah Bebas Duplikat: ', train.shape[0])

Jumlah Awal Data:  500
Jumlah Setelah Bebas Duplikat:  500


In [10]:
train['processed_text'] = train['Review Produk'].apply(preprocess_text)
train.head()  

,ID Data,Nama Produk,Jenis Kulit,Review Produk,Rating,Label,processed_text
0,A001,Cetaphil: Gentle Skin Cleanser,Oily,Aku suka banget sama ini. Cuma ini cleanser ya...,5,Positive,aku suka banget sama ini cuma ini cleanser ya...
1,A002,Cetaphil: Gentle Skin Cleanser,Oily,Walaupun produknya mengklaim dapat digunakan s...,5,Positive,walaupun produknya mengklaim dapat digunakan s...
2,A003,Cetaphil: Gentle Skin Cleanser,Dry,Thanks God for this product. Produk ini udah p...,5,Positive,thanks god for this product produk ini udah p...
3,A004,Cetaphil: Gentle Skin Cleanser,Oily,Another favorite product from Cetaphil. Seneng...,5,Positive,another favorite product from cetaphil seneng...
4,A005,Cetaphil: Gentle Skin Cleanser,Oily,awalnya kaget karena cleanser ini enggak berbu...,5,Positive,awalnya kaget karena cleanser ini enggak berbu...


In [11]:
train.Label.value_counts()

Label
Positive    250
Negative    250
Name: count, dtype: int64

In [12]:
possible_labels = train.Label.unique()

In [13]:
possible_labels = sorted(possible_labels)

In [14]:
possible_labels

['Negative', 'Positive']

In [15]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [16]:
label_dict

{'Negative': 0, 'Positive': 1}

In [17]:
train.Label = train['Label'].map(label_dict)

In [18]:
train

,ID Data,Nama Produk,Jenis Kulit,Review Produk,Rating,Label,processed_text
0,A001,Cetaphil: Gentle Skin Cleanser,Oily,Aku suka banget sama ini. Cuma ini cleanser ya...,5,1,aku suka banget sama ini cuma ini cleanser ya...
1,A002,Cetaphil: Gentle Skin Cleanser,Oily,Walaupun produknya mengklaim dapat digunakan s...,5,1,walaupun produknya mengklaim dapat digunakan s...
2,A003,Cetaphil: Gentle Skin Cleanser,Dry,Thanks God for this product. Produk ini udah p...,5,1,thanks god for this product produk ini udah p...
3,A004,Cetaphil: Gentle Skin Cleanser,Oily,Another favorite product from Cetaphil. Seneng...,5,1,another favorite product from cetaphil seneng...
4,A005,Cetaphil: Gentle Skin Cleanser,Oily,awalnya kaget karena cleanser ini enggak berbu...,5,1,awalnya kaget karena cleanser ini enggak berbu...
...,...,...,...,...,...,...,...
495,A496,Acnes: Creamy Wash,Dry,Aku beli ini karna baca blog female daily 3 cl...,1,0,aku beli ini karna baca blog female daily 3 cl...
496,A497,Acnes: Creamy Wash,Dry,"aku pake ini udh ngabisin 1 botol, saat aku je...",1,0,aku pake ini udh ngabisin 1 botol saat aku je...
497,A498,Acnes: Creamy Wash,Dry,Facial Wash yg aku beli pas awal2 mulai jerawa...,1,0,facial wash yg aku beli pas awal2 mulai jerawa...
498,A499,Acnes: Creamy Wash,Dry,Aku pernah beli ini dengan harapan bisa bantu ...,1,0,aku pernah beli ini dengan harapan bisa bantu ...


In [19]:
from sklearn.model_selection import train_test_split

# Bagi dataset menjadi train+validation (80%) dan test (20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    train['processed_text'].values,
    train['Label'].values,
    test_size=0.2,
    stratify=train['Label'].values,
    random_state=42
)

#  Bagi train+validation menjadi train (70%) dan validation (10%)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_val,
    y_train_val,
    test_size=0.125,  # 0.125 dari 80% = 10% dari keseluruhan data
    stratify=y_train_val,
    random_state=42
)


In [20]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("indobenchmark/indobert-lite-large-p1")


/usr/local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [21]:
def tokenize(data, max_len):
    input_ids = []
    attention_masks = []
    for text in data:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='np'
        )
        input_ids.append(encoded['input_ids'][0])
        attention_masks.append(encoded['attention_mask'][0])
    
    return np.array(input_ids), np.array(attention_masks)

In [22]:
train_input_ids, train_attention_masks = tokenize(X_train, 128)
val_input_ids, val_attention_masks = tokenize(X_valid, 128)
test_input_ids, test_attention_masks = tokenize(X_test, 128)

In [23]:
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)

In [24]:
class_weights

array([1., 1.])

In [25]:
ohe = preprocessing.OneHotEncoder()
y_train = ohe.fit_transform(y_train.reshape(-1,1)).toarray()
y_valid = ohe.transform   (y_valid.reshape(-1,1)).toarray()
y_test  = ohe.transform   (y_test .reshape(-1,1)).toarray()

# Simpan urutan nama kelas dari OHE agar konsisten di semua laporan/plot
class_names = ohe.categories_[0].tolist()


In [26]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("TPU ready.")
except Exception as e:
    print("No TPU, fallback to default strategy:", e)
    strategy = tf.distribute.get_strategy()


No TPU, fallback to default strategy: Please provide a TPU Name to connect to.


In [27]:
print(tokenizer.__class__.__name__)   # harusnya 'AlbertTokenizerFast'
print(tokenizer("coba tokenisasi cepat")["input_ids"][:10])


BertTokenizerFast
[2, 1841, 17461, 1144, 972, 3]


In [28]:
from transformers import TFAlbertModel  # pastikan import ini aktif (bukan di sel yang lain)

def create_model(max_len=128):
    with strategy.scope():  # ← 'strategy' sesuai Sel 26
        bert_model = TFAlbertModel.from_pretrained(
            "indobenchmark/indobert-lite-large-p1",
            from_pt=True
        )
        opt = tf.keras.optimizers.AdamW(learning_rate=2e-5, weight_decay=1e-2)
        loss = tf.keras.losses.CategoricalCrossentropy()
        metrics = [tf.keras.metrics.CategoricalAccuracy()]

        input_ids       = tf.keras.Input(shape=(max_len,), dtype='int32')
        attention_masks = tf.keras.Input(shape=(max_len,), dtype='int32')

        outputs    = bert_model(input_ids=input_ids, attention_mask=attention_masks)
        embeddings = outputs.pooler_output

        x      = tf.keras.layers.Dropout(0.2)(embeddings)
        output = tf.keras.layers.Dense(2, activation="softmax")(x)

        model = tf.keras.Model([input_ids, attention_masks], output)
        model.compile(optimizer=opt, loss=loss, metrics=metrics)
    return model


In [29]:
model = create_model()
model.summary()

/usr/local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


All PyTorch model weights were used when initializing TFAlbertModel.



All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Model: "model"


__________________________________________________________________________________________________


...

 dense (Dense)               (None, 2)                    2050      ['dropout_4[0][0]']           


Total params: 17686018 (67.47 MB)


Trainable params: 17686018 (67.47 MB)


Non-trainable params: 0 (0.00 Byte)


__________________________________________________________________________________________________


In [30]:
# CELL 31
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss',   # Metric to monitor
#     factor=0.1,           # Factor by which the learning rate will be reduced. new_lr = lr * factor
#     patience=50,           
#     verbose=1             # Verbosity mode.
# )

In [31]:
class_weight = {i: class_weights[i] for i in range(len(class_weights))}

In [32]:

# ===== TRAINING =====
model = create_model(max_len=128)   # pastikan max_len konsisten dengan tokenisasi

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1
)

history_bert = model.fit(
    [train_input_ids, train_attention_masks], y_train,
    validation_data=([val_input_ids, val_attention_masks], y_valid),
    epochs=30,
    batch_size=32,
    class_weight=class_weight,          # dict {kelas: bobot}
    callbacks=[reduce_lr, early_stop],
    verbose=1
)


All PyTorch model weights were used when initializing TFAlbertModel.



All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


Epoch 1/30


 1/11 [=>............................] - ETA: 7:07 - loss: 0.6965 - categorical_accuracy: 0.5625

 2/11 [====>.........................] - ETA: 1:06 - loss: 0.7647 - categorical_accuracy: 0.5312

...

 9/11 [=======================>......] - ETA: 14s - loss: 0.0198 - categorical_accuracy: 0.9965

10/11 [==========================>...] - ETA: 7s - loss: 0.0183 - categorical_accuracy: 0.9969 

11/11 [==============================] - ETA: 0s - loss: 0.0170 - categorical_accuracy: 0.9971


Epoch 15: ReduceLROnPlateau reducing learning rate to 1.9999999949504855e-07.


11/11 [==============================] - 83s 8s/step - loss: 0.0170 - categorical_accuracy: 0.9971 - val_loss: 0.5166 - val_categorical_accuracy: 0.8600 - lr: 2.0000e-06


In [33]:
result = model.predict([test_input_ids, test_attention_masks])

1/4 [======>.......................] - ETA: 35s

2/4 [==============>...............] - ETA: 5s 

3/4 [=====================>........] - ETA: 2s

4/4 [==============================] - ETA: 0s

4/4 [==============================] - 18s 2s/step


In [34]:
y_true_idx = y_test.argmax(1) if y_test.ndim > 1 else y_test
y_pred_idx = result.argmax(1)
                             

In [35]:
class_names_raw = ohe.categories_[0].tolist()             # contoh: [0, 1]
# Kalau kamu punya label_dict = {'negative':0, 'positive':1}:
inv_label_dict = {v:k for k,v in label_dict.items()}      # {0:'negative', 1:'positive'}
class_names_str = [str(inv_label_dict.get(x, x)) for x in class_names_raw]  # ['negative','positive'] atau string lain


In [36]:
# ===== CLASSIFICATION REPORT (tabel teks) =====
print(classification_report(
    y_true_idx, y_pred_idx,
    labels=labels_idx,               # urutan harus selaras
    target_names=class_names_str, # wajib string
    digits=4, 
    zero_division=0
))

NameError: name 'classification_report' is not defined

In [ ]:
model.save("/kaggle/working/save_model/promt_model.h5", save_format="h5")


In [ ]:
# ---- Load (H5)
from transformers import TFAlbertModel
model2 = tf.keras.models.load_model(
    "/kaggle/working/save_model/promt_model.h5",
    custom_objects={"TFAlbertModel": TFAlbertModel},
    compile=False
)

In [ ]:
from transformers import TFAlbertModel

# Muat model dengan custom_objects
model2 = tf.keras.models.load_model(
    "/kaggle/working/save_model/promt_model.h5",
    custom_objects={"TFAlbertModel": TFAlbertModel}
)

In [ ]:
promt = "sunblock cukup mahal buat seukuran ini. spf nya sih katanya 45, tapi masih gosong aja pake ini walau udah retouch. gak waterproof gak sweatproof. greasy. dipake nya lumayan cepet menyerap ke kulit. pas panas-panasan kulit jadi lengket gak jelas. clog pores juga. big no bua lt dipake di muka."

In [ ]:
# CELL 50
processed_promt = preprocess_text(promt)

In [ ]:
prom_input_ids, prom_attention_masks = tokenize([processed_promt], 128)

In [ ]:
promt_class = model2.predict([prom_input_ids, prom_attention_masks])

In [ ]:
# Nama kelas
class_names = ['negative', 'positive']

# Prediksi probabilitas
probabilities = promt_class[0]  # Ambil prediksi untuk instance pertama (hanya 1 input)

# Indeks kelas dengan probabilitas tertinggi
predicted_index = np.argmax(probabilities)

# Nama kelas
predicted_class_name = class_names[predicted_index]

# Probabilitas kelas yang diprediksi
predicted_probability = probabilities[predicted_index]

print(f"Predicted Class: {predicted_class_name}")
print(f"Probability: {predicted_probability:.4f}")

In [ ]:
# CELL 54
# from transformers import TFAutoModel, AutoTokenizer
# tokenizer  = AutoTokenizer.from_pretrained("indobenchmark/indobert-lite-large-p1")
# bert_model = TFAutoModel.from_pretrained("indobenchmark/indobert-lite-large-p1", from_pt=True)


In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# --- 1A. TABEL HISTORY ---
hist = history_bert.history
df_hist = pd.DataFrame(hist)
display(df_hist.head())   # tampilan tabel
# Simpan jika perlu
df_hist.to_csv("training_history.csv", index=False)

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# --- 1B. KURVA LOSS ---
plt.figure()
plt.plot(hist.get("loss", []), label="loss")
plt.plot(hist.get("val_loss", []), label="val_loss")
plt.title("Kurva Loss (Train vs Validasi)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# --- 1C. KURVA AKURASI (nama metrik bisa 'categorical_accuracy' atau 'accuracy') ---
acc_key = "categorical_accuracy" if "categorical_accuracy" in hist else "accuracy"
val_acc_key = f"val_{acc_key}"
if acc_key in hist and val_acc_key in hist:
    plt.figure()
    plt.plot(hist[acc_key], label=acc_key)
    plt.plot(hist[val_acc_key], label=val_acc_key)
    plt.title("Kurva Akurasi (Train vs Validasi)")
    plt.xlabel("Epoch")
    plt.ylabel("Akurasi")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# --- 1C. KURVA AKURASI (nama metrik bisa 'categorical_accuracy' atau 'accuracy') ---
acc_key = "categorical_accuracy" if "categorical_accuracy" in hist else "accuracy"
val_acc_key = f"val_{acc_key}"
if acc_key in hist and val_acc_key in hist:
    plt.figure()
    plt.plot(hist[acc_key], label=acc_key)
    plt.plot(hist[val_acc_key], label=val_acc_key)
    plt.title("Kurva Akurasi (Train vs Validasi)")
    plt.xlabel("Epoch")
    plt.ylabel("Akurasi")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# CELL 59
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# --- pastikan y_true & y_pred ---
y_true_idx = y_test.argmax(1) if y_test.ndim > 1 else y_test
y_pred_idx = result.argmax(1)  # dari softmax

class_names = ["Negative","Positive"]  # sesuaikan

# --- 2A. TABEL CLASSIFICATION REPORT ---
report = classification_report(
    y_true_idx, y_pred_idx,
    target_names=class_names_str,
    output_dict=True,
    digits=4,
    zero_division=0
)
df_report = pd.DataFrame(report).T
display(df_report)
df_report.to_csv("classification_report.csv", index=False)

# --- 2B. TABEL CONFUSION MATRIX ---
cm = confusion_matrix(y_true_idx, y_pred_idx, labels=list(range(len(class_names_str))))
print("Confusion Matrix (angka):\n", cm)

plt.figure()
plt.imshow(cm, interpolation="nearest")
plt.title("Confusion Matrix")
plt.colorbar()
tick_marks = np.arange(len(class_names_str))
plt.xticks(tick_marks, class_names_str, rotation=45)
plt.yticks(tick_marks, class_names_str)
thresh = cm.max() / 2.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], "d"),
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black")
plt.ylabel("Aktual")
plt.xlabel("Prediksi")
plt.tight_layout()
plt.show()

# --- 2C. GAMBAR CONFUSION MATRIX (tanpa seaborn) ---
bal_acc = balanced_accuracy_score(y_true_idx, y_pred_idx)
print(f"Balanced Accuracy: {bal_acc:.4f}")

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
import numpy as np, matplotlib.pyplot as plt

# Ambil probabilitas kelas positif (kolom 1 untuk binary softmax [neg, pos])
y_true_idx = y_test.argmax(1) if y_test.ndim > 1 else y_test
y_score_pos = result[:, 1]

# --- ROC ---
auc = roc_auc_score(y_true_idx, y_score_pos)
fpr, tpr, _ = roc_curve(y_true_idx, y_score_pos)
plt.figure()
plt.plot(fpr, tpr, label=f"AUC = {auc:.4f}")
plt.plot([0,1],[0,1], linestyle="--")
plt.title("ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(); plt.grid(True); plt.show()

# --- Precision–Recall ---
prec, rec, _ = precision_recall_curve(y_true_idx, y_score_pos)
ap = average_precision_score(y_true_idx, y_score_pos)
plt.figure()
plt.plot(rec, prec, label=f"AP = {ap:.4f}")
plt.title("Precision–Recall Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(); plt.grid(True); plt.show()


In [ ]:
import numpy as np, matplotlib.pyplot as plt

def plot_class_dist(y, title):
    y_idx = y.argmax(1) if y.ndim > 1 else y
    counts = np.bincount(y_idx)
    plt.figure()
    plt.bar(range(len(counts)), counts)
    plt.title(title)
    plt.xlabel("Kelas")
    plt.ylabel("Jumlah")
    plt.xticks(range(len(counts)))
    plt.grid(axis="y")
    plt.show()

plot_class_dist(y_train, "Distribusi Kelas - Train")
plot_class_dist(y_valid, "Distribusi Kelas - Validasi")
plot_class_dist(y_test,  "Distribusi Kelas - Test")


In [ ]:
# CELL 62
# from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score
# import numpy as np

# # ===== CLASSIFICATION REPORT (tabel teks) =====
# print(classification_report(
#     y_true_idx, y_pred_idx, target_names=class_names, digits=4, zero_division=0
# ))

# # ===== CONFUSION MATRIX (angka mentah) =====
# cm = confusion_matrix(y_true_idx, y_pred_idx, labels=range(len(class_names)))
# print("Confusion Matrix:\n", cm)

# # Jika Anda sudah punya helper `conf_matrix(y_true, y_pred, title, class_names)`
# # panggil dengan indeks, bukan one-hot:
# conf_matrix(y_true_idx, y_pred_idx, "Sentiment Analysis — Confusion Matrix", class_names_str)

# # ===== BALANCED ACCURACY =====
# bal_acc = balanced_accuracy_score(y_true_idx, y_pred_idx)
# print(f"Balanced Accuracy: {bal_acc:.4f}")
